# Import and Clean Data

The data must be imported from a CSV file, cleaned to the standards of the question being asked, and adapted into a neural network input.


## References

 * [Andrew Long](https://github.com/andrewwlong/diabetes_readmission/blob/master/diabetes_project.ipynb)
 * [Sheng Weng](https://github.com/swengzju/Predicting-Diabetes-Patient-Readmission/blob/master/Predicting%20Diabetes%20Patient%20Readmission.ipynb)
 * [Bose, et al.](https://github.com/Yawhoong/ISS-Diabetes-Readmission/blob/master/Identification%20of%20Critical%20Risk%20Factors%20leading%20to%20short-term%20readmission%20of%20Diabetic%20Patients.pdf)


In [1]:
# Import modules and data
import pandas as pd
import numpy as np

df = pd.read_csv("diabetic_data.csv")
df.head()

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),?,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),?,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),?,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),?,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),?,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [2]:
# Determine whether the diabetes diagnosis was in the top 3
# Drop the diagnoses, they are too scattered
import re
DIABETES_REGEX = re.compile("250")
isDiabetes = lambda s: DIABETES_REGEX.match(s)
df["primary_diag"] = (df["diag_1"].str.contains("^250") |
                      df["diag_2"].str.contains("^250") |
                      df["diag_3"].str.contains("^250")).astype("int")
df.drop(["diag_1", "diag_2", "diag_3"], axis=1, inplace=True)
df.groupby("primary_diag").size()

primary_diag
0    63742
1    38024
dtype: int64

In [3]:
# Remove columns...
#  * encounter_id, patient_nbr (identification)
#  * weight, payer_code, medical_specialty (too sparse)
#  * diag_1, diag_2, diag_3 (too specific)
df.drop(["encounter_id", "patient_nbr", "weight", "payer_code", "medical_specialty"], axis=1, inplace=True)
df.head()

,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,primary_diag
0,Caucasian,Female,[0-10),6,25,1,1,41,0,1,...,No,No,No,No,No,No,No,No,NO,1
1,Caucasian,Female,[10-20),1,1,7,3,59,0,18,...,Up,No,No,No,No,No,Ch,Yes,>30,1
2,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,13,...,No,No,No,No,No,No,No,Yes,NO,1
3,Caucasian,Male,[30-40),1,1,7,2,44,1,16,...,Up,No,No,No,No,No,Ch,Yes,NO,1
4,Caucasian,Male,[40-50),1,1,7,1,51,0,8,...,Steady,No,No,No,No,No,Ch,Yes,NO,1


In [4]:
# Filter by...
#  * Must have a gender
df = df[df["gender"] != "Unknown/Invalid"]

#  * Must not be discharged by death or hospice
df = df.loc[~df["discharge_disposition_id"].isin([11, 13, 14, 19, 20, 21])]

df.shape

(99340, 43)

In [5]:
# Normalize continuous columns to [0, 1]
cols_continuous = ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient','number_diagnoses']

for col in cols_continuous:
    lo, hi = df[col].min(), df[col].max()
    df[col] = (df[col] - lo) / (hi - lo)

In [6]:
# Normalize numerically-categorical columns (i.e. age)
AGE_MAPPING = {"[{}-{})".format(i * 10, (i + 1) * 10): i * 0.1 + 0.05 for i in range(10)}
df["age"] = df["age"].apply(lambda s: AGE_MAPPING[s])
df[["age"] + cols_continuous].head()

,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses
0,0.05,0.000000,0.305344,0.000000,0.0000,0.000000,0.0,0.000000,0.000000
1,0.15,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.000000,0.533333
2,0.25,0.076923,0.076336,0.833333,0.1500,0.047619,0.0,0.047619,0.333333
3,0.35,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.000000,0.400000
4,0.45,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.000000,0.266667


In [7]:
# Manipulate categorical columns
cols_categorical = ['race', 'gender',
        'admission_type', 'admission_source',
        'max_glu_serum', 'A1Cresult',
        'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
        'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
        'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
        'tolazamide', 'insulin',
        'glyburide-metformin', 'glipizide-metformin',
        'glimepiride-pioglitazone', 'metformin-rosiglitazone',
        'metformin-pioglitazone', 'change', 'diabetesMed',
        'examide', 'citoglipton']

# Create an "Unknown" category for race
df["race"] = df["race"].replace("?", "Unknown")

# Consolidate admissions types
ADMISSION_TYPE_MAPPING = {
    1: "Emergency",
    2: "Urgent",
    3: "Elective"
}
df["admission_type_id"] = df["admission_type_id"].apply(lambda s: ADMISSION_TYPE_MAPPING.get(s, "Other"))

# Discharge disposition is either going home OK or not going home OK
df["discharge_disposition_id"] = (df["discharge_disposition_id"] == 1).astype("int")

# Admission source is either Emergency, Referral, or Other
ADMISSION_SOURCE_MAPPING = {
    1: "EmergencyRoom",
    7: "Referral"
}
df["admission_source_id"] = df["admission_type_id"].apply(lambda s: ADMISSION_SOURCE_MAPPING.get(s, "Other"))

df.rename(columns={
    "admission_type_id": "admission_type",
    "discharge_disposition_id": "discharged_home",
    "admission_source_id": "admission_source"}, inplace=True)
#df["race", "admission_type", "admission_source", "discharged_home"].head()
df.head()

,race,gender,age,admission_type,discharged_home,admission_source,time_in_hospital,num_lab_procedures,num_procedures,num_medications,...,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted,primary_diag
0,Caucasian,Female,0.05,Other,0,Other,0.000000,0.305344,0.000000,0.0000,...,No,No,No,No,No,No,No,No,NO,1
1,Caucasian,Female,0.15,Emergency,1,Other,0.153846,0.442748,0.000000,0.2125,...,Up,No,No,No,No,No,Ch,Yes,>30,1
2,AfricanAmerican,Female,0.25,Emergency,1,Other,0.076923,0.076336,0.833333,0.1500,...,No,No,No,No,No,No,No,Yes,NO,1
3,Caucasian,Male,0.35,Emergency,1,Other,0.076923,0.328244,0.166667,0.1875,...,Up,No,No,No,No,No,Ch,Yes,NO,1
4,Caucasian,Male,0.45,Emergency,1,Other,0.000000,0.381679,0.000000,0.0875,...,Steady,No,No,No,No,No,Ch,Yes,NO,1


In [8]:
# Handle categorical columns...
#  1. Remove categorical columns that are almost uniform (>99% of entries are one thing)
#  2. Split other columns into multiple boolean columns
n_rows = df.shape[0]
cols_insignificant = []
cols_significant = []
for col in cols_categorical:
    freqs = dict(df.groupby(col).size())
    
    # Remove this column
    if any([float(value)/float(n_rows) > 0.99 for value in freqs.values()]):
        cols_insignificant.append(col)
    
    else:
        # Sanity check
        assert(len(freqs) > 1)
        cols_significant.append(col)
        
        # Convert to column to string if not already string type
        if df[col].dtype != str:
            df[col] = df[col].astype(str)
            
# Extract output
df_output = pd.get_dummies(df["readmitted"])
df.drop(["readmitted"], axis=1, inplace=True)

df_cat = pd.get_dummies(df[cols_significant], drop_first=False)
df_input = pd.concat([df, df_cat], axis=1)

df_input.drop(cols_insignificant + cols_significant, axis=1, inplace=True)

print("Insignificant: {}".format(cols_insignificant))
print("Significant: {}".format(cols_significant))
df_input.head()

Insignificant: ['admission_source', 'nateglinide', 'chlorpropamide', 'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 'examide', 'citoglipton']
Significant: ['race', 'gender', 'admission_type', 'max_glu_serum', 'A1Cresult', 'metformin', 'repaglinide', 'glimepiride', 'glipizide', 'glyburide', 'pioglitazone', 'rosiglitazone', 'insulin', 'change', 'diabetesMed']


,age,discharged_home,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,...,rosiglitazone_Steady,rosiglitazone_Up,insulin_Down,insulin_No,insulin_Steady,insulin_Up,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes
0,0.05,0,0.000000,0.305344,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,0,0,0,1,0,0,0,1,1,0
1,0.15,1,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.000000,0.533333,...,0,0,0,0,0,1,1,0,0,1
2,0.25,1,0.076923,0.076336,0.833333,0.1500,0.047619,0.0,0.047619,0.333333,...,0,0,0,1,0,0,0,1,0,1
3,0.35,1,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.000000,0.400000,...,0,0,0,0,0,1,1,0,0,1
4,0.45,1,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.000000,0.266667,...,0,0,0,0,1,0,1,0,0,1


At this point, all cells in the data should be numeric between 0 and 1. There are three different outcomes represented. Outputs are left in the same table as inputs to ensure that they do not get scrambled.

In [9]:
df = pd.concat([df_input, df_output], axis=1)
df.rename(columns={"<30": "OUTPUT_<30", ">30": "OUTPUT_>30", "NO": "OUTPUT_NO"}, inplace=True)
df.head()

,age,discharged_home,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,...,insulin_No,insulin_Steady,insulin_Up,change_Ch,change_No,diabetesMed_No,diabetesMed_Yes,OUTPUT_<30,OUTPUT_>30,OUTPUT_NO
0,0.05,0,0.000000,0.305344,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,...,1,0,0,0,1,1,0,0,0,1
1,0.15,1,0.153846,0.442748,0.000000,0.2125,0.000000,0.0,0.000000,0.533333,...,0,0,1,1,0,0,1,0,1,0
2,0.25,1,0.076923,0.076336,0.833333,0.1500,0.047619,0.0,0.047619,0.333333,...,1,0,0,0,1,0,1,0,0,1
3,0.35,1,0.076923,0.328244,0.166667,0.1875,0.000000,0.0,0.000000,0.400000,...,0,0,1,1,0,0,1,0,0,1
4,0.45,1,0.000000,0.381679,0.000000,0.0875,0.000000,0.0,0.000000,0.266667,...,0,1,0,1,0,0,1,0,0,1


# Neural Network

Bose, et al. used a neural network with the following parameters to model the given data:

 * Layers:
    1. Input (size = 70)
    2. Hidden Layer 1 (size = 70, ReLU activation)
    3. Hidden Layer 2 (size = 20, ReLU activation)
    4. Output Layer (size = 2, softmax)  
 * Dropout Rate: 0.1
 * Batch Size: 100
 * Epoch Size: 50
 * Optimization Algorithm: Adaptive Moment Estimation
 * Cost Function: Optimizes for recall
 
[This tutorial](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py) is useful for implementing a basic multi-layer perceptron classifier.

In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Hyperparameters
N_INPUTS = 67

## Data Separation

The data is separated into training, validation, and testing pools using a random number generator always seeded with the same value (so separation occurs the same way every time)

In [11]:
# Pull out validation (15%) and test (15%) data from training (70%) data
df_valid_test = df.sample(frac=0.3, random_state=0xda)
df_training = df.drop(df_valid_test.index)

df_valid = df_valid_test.sample(frac=0.5, random_state=0xdb)
df_test = df_valid_test.drop(df_valid.index, axis=0)

df_training.shape[0], df_valid.shape[0], df_test.shape[0]

(69538, 14901, 14901)

## Batches

The following class can be used as an iterable in a `for` loop. It takes a dynamic collection of data separated into pools (e.g. categories like readmission vs. no readmission) and produces a batch which samples equally frequently from each pool. The iterable stops when all data in the largest pool have been iterated over once

In [12]:
class TrainingBatches:

    def __init__(self, nn, batch_size=16):
        self.nn = nn
        self.batch_size = batch_size
        self.bounds = max([d.shape[0] for d in nn.training_data])
        self.it = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.it >= self.bounds:
            raise StopIteration

        output = np.concatenate([self.getSlice(d, self.it, self.batch_size) for d in self.nn.training_data])
        self.it += self.batch_size
        return output[:, :N_INPUTS], output[:, N_INPUTS:]

    # Code must work with Python 2 and 3
    next = __next__

    def getSlice(self, array, start, length):
        start = start % array.shape[0]
        if start + length >= array.shape[0]:
            return np.concatenate((array[start:], array[:(start + length) % array.shape[0]]), axis=0)
        else:
            return array[start:start + length]

In [13]:
class CategoricalPreprocessor:
    
    def __init__(self, df, output_categories):
        self.output_categories = output_categories
        self.df = df
        self.is_modified = False
        
    def modifyDatafile(self):
        
        if self.is_modified:
            return self.df
        
        self.df = self.df.copy()
        self.is_modified = True
        
        if self.output_categories == "any":
            self.df["OUTPUT_ANY"] = self.df["OUTPUT_<30"] + self.df["OUTPUT_>30"]
            self.df.drop(["OUTPUT_<30", "OUTPUT_>30"], axis=1, inplace=True)

        elif self.output_categories == "rapid":
            self.df["OUTPUT_NO"] = self.df["OUTPUT_>30"] + self.df["OUTPUT_NO"]
            self.df.drop(["OUTPUT_>30"], axis=1, inplace=True)
            
        return self.df
    
    def getArraysByOutput(self):
        
        if not self.is_modified:
            self.modifyDatafile()
            
        if self.output_categories == "three":
            return (
                self.df[self.df["OUTPUT_<30"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_>30"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy()
            )

        elif self.output_categories == "any":
            return (
                self.df[self.df["OUTPUT_ANY"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy()
            )

        else:
            return (
                self.df[self.df["OUTPUT_<30"] == 1].to_numpy(),
                self.df[self.df["OUTPUT_NO"] == 1].to_numpy()
            )

## Neural Network Generator

The following class dynamically generates a neural network with its own training and validation data

In [29]:
class NeuralNetwork:

    def __init__(self, df, df_validation, layer_sizes, output_categories, dropout, batch_size):
        self.dropout = dropout
        self.output_categories = output_categories
        self.batch_size = batch_size
        self.layer_sizes = layer_sizes
        
        # Parameter to horizontally compress the sigmoid activation
        self.activation_param = 1.0
        
        # Prepare training data
        training_processor = CategoricalPreprocessor(df, output_categories)
        self.training_data = training_processor.getArraysByOutput()
        
        # Prepare training batches
        training_in_out = list(iter(TrainingBatches(self, 32)))
        self.training_in = np.concatenate([i for i, o in training_in_out])
        self.training_out = np.concatenate([o for i, o in training_in_out])
        
        # Prepare validation data
        validation_processor = CategoricalPreprocessor(df_validation, output_categories)
        self.validation_data = validation_processor.modifyDatafile().to_numpy()
        self.validation_in = self.validation_data[:, :N_INPUTS]
        self.validation_out = self.validation_data[:, N_INPUTS:]
        
        self.classifier = self.makeClassifier()
        
        
    def parametrizedSigmoid(self):
        
        def sigmoid(x):
            # In the original Tensorflow code, x is a Tensor
            # For efficiency, x will be scalar-multiplied by the sigmoid
            # parameter and then passed to tf.math.sigmoid
            scaled = tf.math.scalar_mul(self.activation_param, x)
            return tf.math.sigmoid(scaled)
        
        return sigmoid
    
    
    def makeClassifier(self):
        
        # Create layers
        classifier = tf.keras.Sequential()
        classifier.add(tf.keras.layers.Dense(
            self.layer_sizes[0], activation="sigmoid", kernel_initializer="random_normal", input_dim=N_INPUTS))
        classifier.add(tf.keras.layers.Dropout(self.dropout))
        for size in self.layer_sizes[1:]:
            classifier.add(tf.keras.layers.Dense(
                size, activation="sigmoid", kernel_initializer="random_normal"))
            classifier.add(tf.keras.layers.Dropout(self.dropout))
        classifier.add(tf.keras.layers.Dense(
            len(self.training_data), activation=self.parametrizedSigmoid(), kernel_initializer="random_normal"))
        
        # Compile model
        classifier.compile(optimizer="adam", loss="binary_crossentropy",
                                metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])
        
        return classifier
        
        
    def train(self, epochs=20):
        return self.classifier.fit(self.training_in, self.training_out,
                                   validation_data=(self.validation_in, self.validation_out),
                                   batch_size=self.batch_size,
                                   epochs=epochs)

    
    def steepen(self):
        # Preserve old parameters
        params = [layer.get_weights() for layer in self.classifier.layers if type(layer) == tf.keras.layers.Dense]
        
        self.activation_param *= 2
        self.classifier = self.makeClassifier()
        
        for layer, param in zip([layer for layer in self.classifier.layers if type(layer) == tf.keras.layers.Dense], params):
            layer.set_weights(param)
    
    
    def configStr(self):
        return "Layers: {}; Output Categories: {}; Dropout: {}; Batch Size: {}".format(
            self.layer_sizes, self.output_categories, self.dropout, self.batch_size)

In [30]:
nn = NeuralNetwork(**{
    "df": df_training.copy(),
    "df_validation": df_valid.copy(),
    "layer_sizes": (70, 20),
    "output_categories": "three",
    "dropout": 0.1,
    "batch_size": 64})

In [21]:
type(nn.classifier.layers[0]) == tf.keras.layers.Dense

True

In [20]:
nn.classifier.layers

In [31]:
for i in range(5):
    nn.train(3)
    nn.steepen()
nn.train(10)

Train on 110208 samples, validate on 14901 samples
Epoch 1/3
110208/110208 [==============================] - 8s 74us/sample - loss: 0.6332 - acc: 0.6650 - auc: 0.5470 - val_loss: 0.6359 - val_acc: 0.6668 - val_auc: 0.5430
Epoch 2/3
110208/110208 [==============================] - 8s 72us/sample - loss: 0.6215 - acc: 0.6700 - auc: 0.5993 - val_loss: 0.6133 - val_acc: 0.6685 - val_auc: 0.6380
Epoch 3/3
110208/110208 [==============================] - 8s 72us/sample - loss: 0.6160 - acc: 0.6730 - auc: 0.6155 - val_loss: 0.6113 - val_acc: 0.6689 - val_auc: 0.6486
Train on 110208 samples, validate on 14901 samples
Epoch 1/3
110208/110208 [==============================] - 8s 75us/sample - loss: 0.6141 - acc: 0.6734 - auc: 0.6219 - val_loss: 0.6109 - val_acc: 0.6702 - val_auc: 0.6444
Epoch 2/3
110208/110208 [==============================] - 8s 73us/sample - loss: 0.6127 - acc: 0.6742 - auc: 0.6247 - val_loss: 0.6052 - val_acc: 0.6802 - val_auc: 0.6531
Epoch 3/3
110208/110208 [=============

In [32]:
nn.activation_param

32.0